# Auto-completion of text using N-gram Language Model

Auto-completion (sometimes known as "word prediction") refers to the prediction of words typed by a user. It has various applications including web browsers, search engines, email programs, word processors, etc. It helps in increasing the typing speed of the user thus saving time and boosting productivity.

A Language Model is a probability distribution over a set of word sequences. These models rely on training data which contain text corpora of large, structured text sequences that can help in finding the probability of a word given a particular sequence preceding it. A Language Model can help predict the next word based on its probability given a sequence of previous words.

This paper focuses on the implementation of the statistical n-gram model for calculating the probability of the next word for implementing auto-completion given a previous sequence of words.
This paper aims to achieve the following objectives -

1. Choosing a Dataset and Obtaining Data
   * Import Libraries
   * Load the dataset
2. Pre-processing data
   * Split the data into sentences
   * Tokenize the sentences into words
3. Splitting the data into train & test sets
4. Text Normalization Step
    * Find the top 10 most frequent words in the set
    * Generate a closed vocabulary list
    * Normalize the train set with \<UNK\> token
5. Building the n-gram model
    * Why n-gram model?
    * Estimation of probabilities
    * Smoothing
6. Testing the n-gram model
    * Input of test sequence
    * Input of first letter of output (Optional)
    * n-gram frequency matrix
    * n-gram probability matrix
    * Word prediction    
7. Evaulating the n-gram model
   * Calculating the Perplexity

## Part A & B: Choosing a Dataset, Obtaining Data

The dataset chosen for this notebook is the Twitter Dataset (**https://www.kaggle.com/datasets/crmercado/tweets-blogs-news-swiftkey-dataset-4million**) containing more than 6 million words in a text file in string form. This dataset was chosen because it provides a large corpus of structured text to form sequences of words and adequate for division into train and test sets. The text file is easy to obtain from the website and is included in the project.

### **Importing Libraries & Loading the Dataset**
Import Python libraries pandas and numpy to organize data and perform analysis. Import src directory containing functions that help in loading, pre-processing, prediction and evaulation of the dataset.

In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')
import sys
import os
import src
import src.preproc as preproc
import src.traintestsplit as traintestsplit
import src.wordvocabulary as wordvocabulary
import src.ngram as ngram
import src.perplexity as perplexity
import src.prediction as prediction

The dataset is loaded by opening the text file and reading its contents in the preproc file.

In [2]:
print("Loading Twitter Data...")
tweets = preproc.load_data()
number_of_tweets = tweets.count('\n')
if (tweets !=""):
    print("Twitter data loaded successfully...")
    print("\nData type:", type(tweets))
    print("\nThere are", number_of_tweets, "tweets in the dataset.")
    number_of_words = len(tweets.split())
    print("There are " + str(number_of_words) + " words in the dataset.")

Loading Twitter Data...
Twitter data loaded successfully...

Data type: <class 'str'>

There are 47961 tweets in the dataset.
There are 616296 words in the dataset.


## Part C: Pre-processing data

The data loaded from the Twitter dataset requires some pre-processing before it can be used for sentence auto-completion.

### Split the data into sentences

First, the data is split into individual sentences and trimmed to remove whitespaces.

In [3]:
sentences = preproc.splitSentences(tweets)
if (sentences!=""):
    print("Tweets split succesfully...")

Tweets split succesfully...


The sentences are displayed using a Pandas Dataframe.

In [4]:
tweets_df = pd.DataFrame(sentences, columns=['TWEETS'])
pd.set_option("display.max_colwidth", None)
tweets_df.head()

,TWEETS
0,"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long."
1,When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
2,they've decided its more fun if I don't.
3,So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)
4,Words from a complete stranger! Made my birthday even better :)


### Tokenize the sentences into words

Next, the sentences are all changed to lowercase and split into words. NLTK's punkt tokenizer is used for this purpose. The result which is a list of words for each tweet is stored in a list.

In [5]:
tokenized_sentences = preproc.tokenizeSentences(sentences)
if (len(tokenized_sentences) !=0):
    print("\nTweets tokenized successfully")
    print("\nTweet before tokenization:\n", sentences[0])
    print("Tweet after tokenization:\n", tokenized_sentences[0])


Tweets tokenized successfully

Tweet before tokenization:
 How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
Tweet after tokenization:
 ['how', 'are', 'you', '?', 'btw', 'thanks', 'for', 'the', 'rt', '.', 'you', 'gon', 'na', 'be', 'in', 'dc', 'anytime', 'soon', '?', 'love', 'to', 'see', 'you', '.', 'been', 'way', ',', 'way', 'too', 'long', '.']


## Part D: Split data into train & test sets

After pre-processing the data, the data can be split into train set and test set. Here, we keep 80% of the data as train data and 20% as test set.

In [6]:
train_data, test_data = traintestsplit.splitData(tokenized_sentences)
if (len(train_data) != 0 and len(test_data)):
    print("Data split into train and test set successfully...")

Data split into train and test set successfully...


In [7]:
print ("Train set contains " + str(len(train_data) )+ " sentences")
print ("Test set contains " + str(len(test_data)) + " sentences")

Train set contains 38368 sentences
Test set contains 9593 sentences


## **Part E:Text Normalization Step**

A collection of the most frequently occurring words is also known as **closed vocabulary**. The n-gram model can predict the probabilities of the words in test set that are a part of the closed vocabulary list. 

However, there may be words in the test set that do not appear in the closed vocabulary created from the training set. In this case, a mechanism is required to deal with the new or "unknown" words. 

For this purpose, a count threshold **N** is introduced. N is an arbitary small integer depending on which a word may be included or excluded from the closed vocabulary of the training set.

### Find the top 10 most frequent words in the set
In the dataset, the frequency of all words is counted and the top 10 most frequent words are listed. 

In [8]:
word_keys =[]
word_counts = wordvocabulary.countWords(tokenized_sentences)
word_counts_keys = sorted(word_counts, key=word_counts.get, reverse=True)[:20]
type(word_counts_keys)
for word in word_counts_keys:
    if word.isalnum():
        word_keys.append(word)
print("\nTop 10 Most Frequent Words in the Dataset -")
print("\nWord\tCount")
print("-------------")
for word in word_keys[:10]:
    print(word + "\t" + str(word_counts.get(word)))



Top 10 Most Frequent Words in the Dataset -

Word	Count
-------------
the	19036
i	18500
to	15787
a	12370
you	12282
and	8840
it	7879
for	7801
is	7709
in	7678


### Generate a closed vocabulary list

If a word appears at least N or more than N times, then it is included in the closed vocabulary. Now, create a list of closed vocabulary words in the train set with a count threshold value N=2.

In [9]:
high_freq_words = wordvocabulary.getHighFreqWords (train_data, 2)
if len(high_freq_words)!=0:
    print("\nThere are " + str(len(high_freq_words)) + " in the closed vocabulary.")


There are 14782 in the closed vocabulary.


### Normalize the train set with \<UNK\> token

Further, the rest of the words are **"Out of Vocabulary (OOV)"** words. These words have a low frequency in the train set. To deal with OOV words which comprise of low frequency words in the train set and the new words not from the test set, the **\<UNK\>** token  which indicates unknown words is introduced. The OOV words in the train set are replaced with the token. Then the token is processed by the n-gram model as a regular word.

In [10]:
print("Before adding <UNK> token")
train_data_sample_before = train_data[:2]
test_data_sample_before = test_data[0:2]
print("Original Train Data")
print(train_data_sample_before)
print("Original Test Data")
print(test_data_sample_before)
train_data_processed = wordvocabulary.addUnkownToken(train_data, high_freq_words)
test_data_processed = wordvocabulary.addUnkownToken(test_data, high_freq_words)
print("-------------------------------")
print("After adding <UNK> token")
print("Modified Train Data")
print(train_data_processed[0:2])
print("Modified Test Data")
print(test_data_processed[0:2])

Before adding <UNK> token
Original Train Data
[['i', "'m", 'sorry', '.'], ['cause', 'yo', 'junky', 'ass', 'always', 'got', 'change', 'pussy', '!', '...']]
Original Test Data
[['just', 'got', 'to', 'intelligentsia', '-', 'free', 'wifi', ',', 'yea', '!', 'still', 'setting', 'up', ',', 'and', 'i', 'am', 'in', 'the', 'middle', 'of', 'the', 'shop', 'with', 'notebook', 'and', 'webcam', '!'], ['everyone', 'eventually', 'leaves', ',', 'willingly', 'it', 'unwillingly', '.']]
-------------------------------
After adding <UNK> token
Modified Train Data
[['i', "'m", 'sorry', '.'], ['cause', 'yo', '<UNK>', 'ass', 'always', 'got', 'change', 'pussy', '!', '...']]
Modified Test Data
[['just', 'got', 'to', '<UNK>', '-', 'free', 'wifi', ',', 'yea', '!', 'still', 'setting', 'up', ',', 'and', 'i', 'am', 'in', 'the', 'middle', 'of', 'the', 'shop', 'with', 'notebook', 'and', 'webcam', '!'], ['everyone', 'eventually', 'leaves', ',', '<UNK>', 'it', '<UNK>', '.']]


## **Part F: Building the n-gram model**

n-gram simply refers to a sequence of **n** words or characters from a given sentence. n-gram is a classic probabilistic language model assumes that the probability of the next word in a sequence is dependent on previous (n-1) words. The base of this assumption is the  n<sup>th</sup> order Markov assumption. In a Markov model the assumption is that we can forecast the likelihood of a future item in a sequence without considering items too far back in the sequence. For a uni-gram model, it is 1<sup>st</sup> order Markov assumption. Similarly, for bi-gram model, it is 2<sup>nd</sup> order Markov assumption. 


**Why n-gram model?**
- It is a simple model to understand and implement.
- Higher values of n which contain more context with a reasonable space-time tradeoff allow the usage of this model in a small project which may expand later.

For a given previous sequence of words $w_{i-1}, w_{i-2} \cdots w_{i-n}$ for a word at i<sup>th</sup> position, the conditional probability of word $w_{i}$ can be given as -

$$ P(w_i | w_{i-1}\dots w_{i-n}) \tag{1}$$

This probability is nothing but the frequency or count of the given sequence in the train set.

**Estimation of probabilities**

The probabilities for an n-gram can be estimated by **maximum likelihood estimation (MLE)** which involves counting the frequency of the n-gram in the corpus and then normalizing it (dividing by  total count) to fall between 0 and 1. In other words, n-gram probability for a word i in a sequence of words  i, i-1, ..., i-n is a ratio of the frequency of a sequence of words from word 'i' through i-n to frequency of sequence of words i-1 through i-n occur in the train set. The equation is given as -

$$ \hat{P}(w_i | w_{i-1}\dots w_{i-n}) = \frac{C(w_{i-1}\dots w_{i-n}, w_n)}{C(w_{i-1}\dots w_{i-n})} \tag{2} $$

where $C(\cdots)$ function is denotes the number of times of the given sequence occurs.
and $\hat{P}$ denotes estimation of $P$.

For example, for a certain train set we have the following data -

- Test data: "I like tea more than"
- Expected output: "coffee"

So, the probability of the word "coffee" occurring after the given test sequence and value of n as 5 is calculated as 

P("coffee") = Number of times "I like tea more than coffee" in train set **/** Number of times "I like tea more than" in train set

So n-grams probability estimation requires the counts of (n+1)-grams for numerator and the counts of n-grams for denominator.


In the below code, n-grams are calculated for the train data with n value ranging from 1 to 6.

In [11]:
n_gram_counts_list = []
# Computing n-gram counts with n in (1,2,3,4,5)
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = ngram.countNGrams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)
print("Finished computing n-gram counts")

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...
Finished computing n-gram counts


### **Smoothing**

It is noted that this equation would work for a sequence which already occurs in the train set. However, there may be cases where an existing word in the train set is used in a different context in the test set and the expected output does not exist in the train set after the test input (an unknown n-gram). This will render the above probability for it as 0. To avoid this problem, a technique known as **smoothing** or **discounting** is introduced. Smoothing involves assigning some probability to such unseen events. The smoothing technique used in this notebook is **add-k smoothing**. Here, a fraction k which is constant is added to count of n-grams of the numerator and k x |V| is added to the denominator. For unknown n-grams, the probability now becomes 1/|V|. The equation with add-k smoothing is given below -

$$ \hat{P}(w_i | w_{i-1}\dots w_{i-n}) = \frac{C(w_{i-1}\dots w_{i-n}, w_n) + k}{C(w_{i-1}\dots w_{i-n}) + k|V|} \tag{3} $$

where k = constant

and |V| = the number of words in the vocabulary of the train set.

The disadvantage of smoothing is that it may assign higher probabilities to unknown n-grams.

## Part G: Testing the model

Now, it is time to test the model created.

### **Input of test sequence**

The user inputs a sentence by running the next code cell.

In [12]:
input_sentence = input("Please enter a sentence : \n")
# sentences = preproc.split_to_sentences(input_sentence)
tokenized_input = nltk.word_tokenize(input_sentence)
# tokenized_input = get_tokenized_data(input_sentence)
print(tokenized_input)

Please enter a sentence : 
i would like to
['i', 'would', 'like', 'to']


### **Input of first letter of output (Optional)**

Then subsequently user may enter the first letter of the next word optionally. If the user does not want to enter the first letter, they can skip this by pressing 'Enter' in the text box.

In [13]:
next_word_starts_with = input("Please enter the first letter of the next word: \n")
n_gram_counts_list = []
for n in range(1, len(tokenized_input) + 1):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = ngram.countNGrams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)
unique_words = list(set(tokenized_input))
word_prediction = prediction.getWordPredictions(tokenized_input, n_gram_counts_list, high_freq_words, k=0.05,  start_with=next_word_starts_with)

word_prediction_list = list(map(list, word_prediction))

for predicted_word in word_prediction_list:
    tmp_probability = np.format_float_positional(predicted_word[1], trim='-')
    predicted_word[1] = tmp_probability

Please enter the first letter of the next word: 
k
Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...


### **n-gram frequency matrix**

A matrix containing n-grams and their frequences for trigram (n=3) is displayed below. 

In [14]:
sentences = []
sentences.append(tokenized_input)
unique_words = list(set(sentences[0]))
unigram_counts = ngram.countNGrams(sentences, 1)
bigram_counts = ngram.countNGrams(sentences, 2)
trigram_counts = ngram.countNGrams(sentences, 3)
ngram.generateCountMatrix(trigram_counts, unique_words)

,to,would,i,like,<END>,<UNK>
"(<S>, i)",0.0,1.0,0.0,0.0,0.0,0.0
"(like, to)",0.0,0.0,0.0,0.0,1.0,0.0
"(would, like)",1.0,0.0,0.0,0.0,0.0,0.0
"(<S>, <S>)",0.0,0.0,1.0,0.0,0.0,0.0
"(i, would)",0.0,0.0,0.0,1.0,0.0,0.0


### **n-gram probabilities matrix**

A matrix containing the probability of each word and the n-gram for trigram (n=3) is displayed below.

In [15]:
ngram.generateProbabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=0.05)
ngram.generateProbabilityMatrix(trigram_counts, unique_words, k=0.05)

,to,would,i,like,<END>,<UNK>
"(<S>, i)",0.038462,0.807692,0.038462,0.038462,0.038462,0.038462
"(like, to)",0.038462,0.038462,0.038462,0.038462,0.807692,0.038462
"(would, like)",0.807692,0.038462,0.038462,0.038462,0.038462,0.038462
"(<S>, <S>)",0.038462,0.038462,0.807692,0.038462,0.038462,0.038462
"(i, would)",0.038462,0.038462,0.038462,0.807692,0.038462,0.038462


### **Word Prediction**

Finally, a list of words predicted for auto-complete are displayed along in order of their calculated probabilities.

In [16]:
word_prediction_df = pd.DataFrame(word_prediction_list, columns=['Next word', 'Probability'])
word_prediction_df['N-gram'] = range(1, len(word_prediction_df) + 1)
word_prediction_df.sort_values('Probability', inplace=True, ascending=False)
word_prediction_df

,Next word,Probability,N-gram
0,know,0.007786891380161949,1
1,know,0.0036000944287063264,2
2,keep,0.00006491820306413918,3


# Part H: Evaluating the n-gram model

Language Models are evaluated through a metric known as **Perplexity (PP)**. The perplexity of a language model on a test set is the test set's inverse probability normalised by the number of words. As inverse is taken, the lower the perplexity, the higher the conditional probability of the word sequence.
Perplexity can be calculated using the formula given below -

\begin{equation}
 PP(W)= \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i|w_{i-1})}}}
\end{equation}

where 
test set W = w1, w2, ...wN,

length of the sentence = N

### **Calculating Perplexity**

In [17]:
unigram_counts = ngram.countNGrams(sentences, 1)
bigram_counts = ngram.countNGrams(sentences, 2)
unique_words = set(tokenized_input)
perplexity_test = perplexity.getPerplexity(tokenized_input,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=0.05)
print(f"Perplexity calculated: {perplexity_test:.4f}")

Perplexity calculated: 1.1177


In other words, minimising perplexity is equivalent to maximising test set probability of a language model.